#  US Gap Analysis Project - WV Breeding Bird Atlas Data Comparison 
Nathan Tarr and Jessie Jordan
USGS NC Cooperative Fish and Wildlife Research Unit
North Carolina State University
Raleigh, NC

We investigated the agreement between WV Breeding Bird Atlas (2011-2015) and USGS Gap Analysis Project data on the species occupying West Virginia.  We also investigated the species that were reported to eBird between 2011 and 2015.


## Key Results

 - The data suggest that the WVBBA did not detect some species that breed in WV, but only a couple.
 - Only two species were reported to eBird that were neither detected by the WVBBA nor predicted to breed in WV by GAP.
 - WVBBA did not detect any species that were not submitted to eBird or predicted to be present in the state by GAP.
 - The WVBBA detected three species that GAP predicted were absent from the state.
 - GAP predicted that some species breed in WV for which no records turned up in the WVBBA data or eBird: Lark Sparrow, Bewick's Wren, and King Rail.  Making this comparison without considering eBird data, the list was five species long.
 - WVBBA detected 28 species in insufficient quantities for modeling.  Among the list were several wetland and nocturnal species as well as rare and sparsely distributed species.
 
## Conclusions

- Our comparisons suggest that whereas the WVBBA detected nearly every species that breeds within WV, several species were detected in insufficient quantities to produce abundance and distribution maps.  Survey effort may have been insufficient in wetlands, at high elevations, and at night.
- GAP's season designations, "winter" and "summer", are valuable because they could support efforts to identify vagrants and late-migrants from species lists.  However, they are not explicitly defined with boundary dates (start and end weeks), and therefore, the appropriateness of such judgements (observation records vs. range data) is questionable and uncertain in many cases.

## Methods
This document demonstrates a comparison of three species lists:

__Gap Analysis Project__ -- we queried the GAP range data to retrieve a list of species that GAP predicts occur within WV during summer or winter.

__WVBBA__ -- we summarized the species detected during WVBBA point count surveys and other efforts to collect data.

__eBird__ -- we pulled a list of species, and the frequency of detection from the eBird API via the R "rebird package" for May and June of the years from 2011 to 2015.  The table returned had the following columns for WV during the months of May and June and years 2011 to 2015:

"monthQt": month and week (a four week period)

"comName": species common name

"frequency": proportion of times the species was seen in a
specified week

"sampleSize" number of complete eBird checklists submitted for
specified given week.

We aggregated weekly mean frequencies of detection to a single value by calculating their mean.

### WVBBA and GAP
We combined a list of species that GAP predicted to be in WV with the WVBBA data by linking records on common names.  We manually resolved unmatched names due to typographical differences, although very few existed.

In [1]:
import pandas as pd
import repo_functions as fun 
import numpy as np
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 15)

# Read in the spp crosswalk
crosswalk = pd.read_csv(fun.dataDir + "SpeciesLists/WV_GAP_Atlas3.csv", header=0, 
                        names=["common_name", "GAP_sci_name", "WVBBA_sci_name", "GAP_code", "WV_code", 'GAP_modeled',
                               'detected_WV', 'occupancy_map_WV', 'density_map_WV', 'change_map_WV', 'breeding_evid_map_WV', 
                               "GAP_habitat(ha)", "WVBBA_individuals", "WVBBA_points", "WVBBA_rate", "%_of_blocks", 
                               "notes"],
                       dtype={"GAP_sci_name":str, "GAP_code":str, "common_name":str, "GAP_habitat(ha)":float, 
                               "WVBBA_sci_name":str, "%_of_blocks":float, "modeled":int, "WV_code":str, "notes":str})
print("The first 5 records")
crosswalk.head().T

The first 5 records


,0,1,2,3,4
common_name,American Kestrel,Bewick's Wren,Blue Grosbeak,Common Grackle,Common Yellowthroat
GAP_sci_name,Falco sparverius,Thryomanes bewickii,Passerina caerulea,Quiscalus quiscula,Geothlypis trichas
WVBBA_sci_name,Falco sparverius,NaN,Passerina caerulea,Quiscalus quiscula,Geothlypis trichas
GAP_code,bamkex,bbewrx,bblgrx,bcogrx,bcoyex
WV_code,AMKE,NaN,BLGR,COGR,COYE
GAP_modeled,1,1,1,1,1
detected_WV,1,0,1,1,1
occupancy_map_WV,1,0,1,1,1
density_map_WV,0,0,0,1,1
change_map_WV,1,0,1,1,1


#### Removing "winter only" species from GAP list
The above table (crosswalk) was developed using a list of species that inhabit WV during summer or winter.  For our comparison, we needed to remove "winter-only" species that were included because of GAP winter data.

In [2]:
# Bring in list of GAP summer species
GAP_summer = pd.read_csv(fun.dataDir + "SpeciesLists/GAP_birds_in_WV_Summer.txt", delimiter="\t", header=0)

In [3]:
# Drop a species if it is a GAP species not in the GAP summer list and not detected by WVBBA
to_exclude = (crosswalk[(crosswalk["GAP_sci_name"].isin(GAP_summer['strScientificName']) == False) &
                   (crosswalk["%_of_blocks"].isnull() == True)])
crosswalk = crosswalk[crosswalk['common_name'].isin(to_exclude['common_name']) == False]

In [4]:
# Update habitat area estimate for species that also winter in WV
crosswalk = pd.merge(crosswalk, GAP_summer.filter(["strCommonName", "intHa"], axis=1),
                     left_on='common_name', right_on='strCommonName', how='outer')
crosswalk["GAP_habitat(ha)"] = crosswalk["intHa"]
crosswalk.drop(["intHa"], axis=1, inplace=True)

### *Exceptions*
Some species entries have been identified as problematic, so we exluded them from this summary.

__*Solitary Vireo*__ -- this concept is equivalent to Blue-headed Vireo, which is also in the tables.

__*Brewster's Warbler*__ and __*Lawrence's Warbler*__ -- these hybrids were not modeled by GAP.

__*Slate-colored Junco*__ -- equivalent to Dark-eyed Junco, which is also in the tables.

In [5]:
drop = ["Solitary Vireo", "Brewster's Warbler", "Lawrence's Warbler", "Slate-colored Junco"]
crosswalk = crosswalk[crosswalk["common_name"].isin(drop) == False]

### eBird

In [6]:
# Ebird pulled via rebird frequency 2011 to 2015 in WV
eBird =(pd.read_csv(fun.dataDir + "SpeciesLists/ebird_WV_2011_2015.csv")
        [lambda x: x['frequency'] > 0]
        [lambda x: x['comName'].str.contains(' sp.') == False]
        [lambda x: x['monthQt'].str.contains('May|June')]
        [lambda x: x['comName'].str.contains('hybrid') == False]
        [lambda x: x['comName'].str.contains('/') == False]
        .drop(['monthQt'], axis=1)
        .groupby('comName').mean()
        .drop(['sampleSize'], axis=1)
        .rename(columns={'frequency': 'eBird_mean_freq'})
        .reset_index()
        .sort_values(by=['eBird_mean_freq'], ascending=False))
eBird.loc[eBird[eBird['comName'] == 'Black-crowned Night-Heron'].index, 'comName'] = 'Black-crowned Night-heron'
eBird.loc[eBird[eBird['comName'] == 'Eastern Whip-poor-will'].index, 'comName'] = 'Whip-poor-will'
eBird.loc[eBird[eBird['comName'] == 'Eurasian Collared-Dove'].index, 'comName'] = 'Eurasian Collared-dove'
eBird['COMMON_NAME'] = [x.upper() for x in eBird['comName']]

We merged the GAP-WVBBA species list crosswalk with the eBird species list to facilitate comparisons of species lists.

In [7]:
# Merge the eBird, WV, and GAP lists
crosswalk["COMMON"] = [x.upper() for x in crosswalk['common_name']]
df = (pd.merge(crosswalk, eBird, left_on='COMMON', right_on='COMMON_NAME',how='outer')
      .drop(["COMMON", "COMMON_NAME"], axis=1)
      .sort_values(by=['common_name']))
df.to_csv(fun.dataDir + "/SpeciesLists/merged_spp_lists.csv")

## Results

### Non-WVBBA
Species on the eBird and GAP lists, but not detected by WVBBA.

In [8]:
a = (df[(df['detected_WV'] == 0) & 
        (df['GAP_modeled'] == 1) &
        (df['comName'].isnull() == False)]
     .drop(["WVBBA_sci_name", "WVBBA_individuals", "WVBBA_points", "WVBBA_rate", "comName", "%_of_blocks",
           "GAP_code", "WV_code", "notes", "strCommonName"], axis=1)
     .sort_values("GAP_habitat(ha)", ascending=False)
     .set_index(["common_name"]))
a

,GAP_sci_name,GAP_modeled,detected_WV,occupancy_map_WV,density_map_WV,change_map_WV,breeding_evid_map_WV,GAP_habitat(ha),eBird_mean_freq
common_name,,,,,,,,,
Cattle Egret,Bubulcus ibis,1.0,0.0,0.0,0.0,0.0,0.0,1079016.39,0.001170
Marsh Wren,Cistothorus palustris,1.0,0.0,0.0,0.0,0.0,0.0,26677.62,0.000655


### eBird only 
Species on the eBird list, but not GAP or WVBBA lists.

In [9]:
b = (df[(df['detected_WV'] == 0) & 
     (df['GAP_modeled'] == 0) &
     (df['comName'].isnull() == False)]
     .sort_values("eBird_mean_freq", ascending=False)
     .set_index(["comName"]))
print(len(b))
b[["eBird_mean_freq"]]

2


,eBird_mean_freq
comName,
Blackpoll Warbler,0.046135
Bay-breasted Warbler,0.013348


### WVBBA only
Species detected by WVBBA, but not submitted to eBird or predicted present by GAP.

In [10]:
c = (df[(df['detected_WV'] == 1) & 
     (df['GAP_modeled'] == 0) &
     (df['comName'].isnull() == True)])
if len(c) > 0:
    print(c['common_name'])
else:
    print("None")

None


### WVBBA minus GAP
Species detected by WVBBA, but not predicted present by GAP.

In [11]:
c = (df[(df['detected_WV'] == 1) & 
     (df['GAP_modeled'] == 0)])
if len(c) > 0:
    print(c['common_name'])
else:
    print("No")

168     Common Merganser
169               Osprey
170    Spotted Sandpiper
Name: common_name, dtype: object


### GAP only
Species predicted present by GAP, but not detected by WVBBA or submitted to eBird.

In [12]:
d = (df[(df['detected_WV'] == 0) & 
     (df['GAP_modeled'] == 1) &
     (df['comName'].isnull() == True)]
    .set_index(["common_name"])
    .sort_values(["GAP_habitat(ha)"], ascending=False))
if len(d) > 0:
    print(d[["GAP_code", "GAP_sci_name", "GAP_habitat(ha)"]])
else:
    print("No")

              GAP_code          GAP_sci_name  GAP_habitat(ha)
common_name                                                  
Lark Sparrow    blaspx  Chondestes grammacus         34328.16
Bewick's Wren   bbewrx   Thryomanes bewickii          1454.67
King Rail       bkirax        Rallus elegans          1255.32


### GAP minus WVBBA
Species predicted present by GAP, but not detected by WVBBA.  eBird is ignored here.

In [13]:
e = (df[(df['detected_WV'] == 0) & 
     (df['GAP_modeled'] == 1)]
    .set_index(["common_name"])
    .sort_values(["common_name"], ascending=True))
if len(e) > 0:
    print(e[["GAP_code", "GAP_sci_name", "GAP_habitat(ha)"]])
else:
    print("No")

              GAP_code           GAP_sci_name  GAP_habitat(ha)
common_name                                                   
Bewick's Wren   bbewrx    Thryomanes bewickii          1454.67
Cattle Egret    bcaegx          Bubulcus ibis       1079016.39
King Rail       bkirax         Rallus elegans          1255.32
Lark Sparrow    blaspx   Chondestes grammacus         34328.16
Marsh Wren      bmawrx  Cistothorus palustris         26677.62


### GAP species not modeled by WVBBA
GAP species detected by WVBBA, but not in enough numbers for occupancy modeling.

In [14]:
e = (df[(df['detected_WV'] == 1) & 
     (df['GAP_modeled'] == 1) &
       (df['occupancy_map_WV'] == 0)]
    .set_index(["common_name"])
    .sort_values(["common_name"], ascending=True))
if len(e) > 0:
    print(e[["GAP_code", "GAP_sci_name", "GAP_habitat(ha)"]])
    print(str(len(e)) + " total")
else:
    print("No")

                          GAP_code              GAP_sci_name  GAP_habitat(ha)
common_name                                                                  
American Bittern            bambix     Botaurus lentiginosus         92506.14
American Black Duck         babdux             Anas rubripes         39890.07
Barn Owl                    bbanox                 Tyto alba        141250.86
Black-crowned Night-heron   bbcnhx     Nycticorax nycticorax          7736.94
Blue-winged Teal            bbwtex              Anas discors         33558.84
Chuck-will's-widow          bcwwix  Caprimulgus carolinensis        188641.35
Dickcissel                  bdickx           Spiza americana         74512.89
Double-crested Cormorant    bdccox     Phalacrocorax auritus              NaN
Eurasian Collared-dove      beucdx     Streptopelia decaocto          8702.73
Henslow's Sparrow           bhespx      Ammodramus henslowii          7660.08
Hooded Merganser            bhomex     Lophodytes cucullatus    